# Unit 5 - Financial Planning

In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

%matplotlib inline

In [2]:
# Load .env enviroment variables
load_dotenv()

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [3]:
# Set current amount of crypto assets
my_btc = 1.2
my_eth = 5.3

In [4]:
# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=CAD"

In [5]:
# Fetch current BTC price
data = requests.get(btc_url).json()['data']
price_btc = data[list(data.keys())[0]]['quotes']['CAD']['price']

# # Fetch current ETH price
data = requests.get(eth_url).json()['data']
price_eth = data[list(data.keys())[0]]['quotes']['CAD']['price']

# # Compute current value of my crpto
my_btc_value = my_btc * price_btc
my_eth_value = my_eth * price_eth

# Print current crypto wallet balance
print(f"The current value of your {my_btc} BTC is ${my_btc_value:0.2f} CAD")
print(f"The current value of your {my_eth} ETH is ${my_eth_value:0.2f} CAD")

### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [6]:
# Set current amount of shares
my_agg = 200
my_spy = 50

In [7]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(
    key_id=alpaca_api_key,
    secret_key=alpaca_secret_key,
    api_version='v2'
)

In [8]:
# Format current date as ISO format
today = pd.to_datetime("today").isoformat()

# Set the tickers
tickers = ['AGG', 'SPY']

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get current closing prices for SPY and AGG
# (use a limit=1000 parameter to call the most recent 1000 days of data)
df = api.get_barset(tickers, timeframe=timeframe, limit=1000).df

# Preview DataFrame
df.tail()

In [9]:
# Pick AGG and SPY close prices
df_close = df[[('AGG', 'close'), ('SPY', 'close')]].droplevel(1, axis=1)
agg_close_price, spy_close_price = df_close.iloc[-1]

# Print AGG and SPY close prices
print(f"Current AGG closing price: ${agg_close_price}")
print(f"Current SPY closing price: ${spy_close_price}")

In [10]:
# Compute the current value of shares
my_spy_value = my_spy * spy_close_price
my_agg_value = my_agg * agg_close_price

# Print current value of shares
print(f"The current value of your {my_spy} SPY shares is ${my_spy_value:0.2f}")
print(f"The current value of your {my_agg} AGG shares is ${my_agg_value:0.2f}")

### Savings Health Analysis

In [11]:
# Set monthly household income
monthly_income = 12000

# Consolidate financial assets data
my_crypto_value = my_btc_value + my_eth_value
my_shares_value = my_agg_value + my_spy_value

# Create savings DataFrame
df_savings = pd.DataFrame.from_dict(
    {
        'crypto': [my_crypto_value],
        'shares': [my_shares_value],
    },
    orient='index',
    columns=['amount']
)

# Display savings DataFrame
print(df_savings.head())

In [12]:
# Plot savings pie chart
df_savings.plot.pie(title='Composition of Personal Savings', y='amount')

In [13]:
# Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
total_savings = df_savings['amount'].sum()

# Validate saving health
if total_savings > emergency_fund:
    message = 'Congratulations! You have enough money in your emergency fund.'
elif total_savings == emergency_fund:
    message = 'Congratulations! You have reached your emergency fund financial goal.'
else:
    message = f'You are still ${round(emergency_fund - total_savings, 2)} away from reaching your emergency fund financial goal.'
print(message)

## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [14]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
date_today = pd.to_datetime("today").tz_localize('Australia/Sydney').tz_convert(None)
date_5yrs_ago = (date_today - pd.DateOffset(years=5))
start_date = date_5yrs_ago.isoformat()
end_date = date_today.isoformat()

In [15]:
# Get 5 years' worth of historical data for SPY and AGG
# (use a limit=1000 parameter to call the most recent 1000 days of data)

# Pull data and append to a list then concatenate at the end for better efficiency
api_limit = 1000
to_date = end_date
stock_data_list = []
# Pull data and drop invalid rows
historical_data = api.get_barset(
    tickers,
    timeframe=timeframe,
    limit=api_limit,
    start=start_date,
    end=to_date,
).df.dropna()
# Keep pulling data until you get all the way to the start date
while historical_data.index[0].isoformat() > start_date:
    # Append all rows except the first row as that will be added in the next iteration
    stock_data_list.append(historical_data.iloc[1:].copy())
    # update the ending date to pull older data
    to_date = historical_data.index[0].isoformat()
    # TODO: Add a delay to obey the rate limit of 200 requests every minute per API key
    historical_data = api.get_barset(
        tickers,
        timeframe=timeframe,
        limit=api_limit,
        start=start_date,
        end=to_date,
    ).df.dropna()
stock_data_list.append(historical_data.copy())

# Concatenate all the data into a single dataframe and sort the index
df_stock_data = pd.concat(stock_data_list).sort_index()
# Select only the data between the start and end dates
df_stock_data = df_stock_data.loc[start_date:end_date]
# Drop duplicate rows if necessary
if True in (df_stock_data.duplicated().value_counts().index):
    df_stock_data.drop_duplicates(inplace=True)
    print('Dropped duplicate rows.')

# Display sample data
df_stock_data.head()

In [16]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
num_yrs_forecast = 30
MC_30_yrs = MCSimulation(
    portfolio_data = df_stock_data,
    weights = [0.4, 0.6],
    num_simulation = 500,
    num_trading_days = 252*num_yrs_forecast
)

In [17]:
# Printing the simulation input data
MC_30_yrs.portfolio_data.head()

In [18]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
MC_30_yrs.calc_cumulative_return()

In [19]:
# Plot simulation outcomes
line_plot_30_yrs = MC_30_yrs.plot_simulation()

In [20]:
# Plot probability distribution and confidence intervals
dist_plot_30_yrs = MC_30_yrs.plot_distribution()

### Retirement Analysis

In [21]:
# Fetch summary statistics from the Monte Carlo simulation results
tbl_30_yrs = MC_30_yrs.summarize_cumulative_return()

# Print summary statistics
print(tbl_30_yrs)

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `$20,000` initial investment.

In [22]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower_30_yrs = round(tbl_30_yrs['95% CI Lower']*initial_investment, 2)
ci_upper_30_yrs = round(tbl_30_yrs['95% CI Upper']*initial_investment, 2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower_30_yrs} and ${ci_upper_30_yrs}.")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [23]:
# Set initial investment
initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000
ci_lower_30_yrs = round(tbl_30_yrs['95% CI Lower']*initial_investment, 2)
ci_upper_30_yrs = round(tbl_30_yrs['95% CI Upper']*initial_investment, 2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower_30_yrs} and ${ci_upper_30_yrs}.")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [24]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns
# Adjust the portfolio to have a higher stock to bond ratio (higher risk)
num_yrs_forecast = 5
MC_5_yrs = MCSimulation(
    portfolio_data = df_stock_data,
    weights = [0.3, 0.7],
    num_simulation = 500,
    num_trading_days = 252*num_yrs_forecast
)

In [25]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
MC_5_yrs.calc_cumulative_return()

In [26]:
# Plot simulation outcomes
line_plot_5_yrs = MC_5_yrs.plot_simulation()

In [27]:
# Plot probability distribution and confidence intervals
dist_plot_5_yrs = MC_5_yrs.plot_distribution()

In [28]:
# Fetch summary statistics from the Monte Carlo simulation results
tbl_5_yrs = MC_5_yrs.summarize_cumulative_return()

# Print summary statistics
print(tbl_5_yrs)

In [29]:
# Set initial investment
initial_investment = 60000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
ci_lower_5_yrs = round(tbl_5_yrs['95% CI Lower']*initial_investment, 2)
ci_upper_5_yrs = round(tbl_5_yrs['95% CI Upper']*initial_investment, 2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_5_yrs} and ${ci_upper_5_yrs}.")

### Ten Years Retirement Option

In [30]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns
num_yrs_forecast = 10
MC_10_yrs = MCSimulation(
    portfolio_data = df_stock_data,
    weights = [0.3, 0.7],
    num_simulation = 500,
    num_trading_days = 252*num_yrs_forecast
)

In [31]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns
MC_10_yrs.calc_cumulative_return()

In [32]:
# Plot simulation outcomes
line_plot_10_yrs = MC_10_yrs.plot_simulation()

In [33]:
# Plot probability distribution and confidence intervals
dist_plot_10_yrs = MC_10_yrs.plot_distribution()

In [34]:
# Fetch summary statistics from the Monte Carlo simulation results
tbl_10_yrs = MC_10_yrs.summarize_cumulative_return()

# Print summary statistics
print(tbl_10_yrs)

In [35]:
# Set initial investment
initial_investment = 60000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
ci_lower_10_yrs = round(tbl_10_yrs['95% CI Lower']*initial_investment, 2)
ci_upper_10_yrs = round(tbl_10_yrs['95% CI Upper']*initial_investment, 2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_10_yrs} and ${ci_upper_10_yrs}.")